In [1]:
import opensim as osim 

In [2]:
model = osim.Model()
model.setName("n_linkage")
model.setUseVisualizer(True)
model.setGravity(osim.Vec3(0, -9.8065, 0));

N_LINKS = 5

In [3]:
links = []

ground =  model.updGround()

for i in range(N_LINKS):
    link = osim.Body(f"link_{i}",
                    1.0,
                    osim.Vec3(0),
                    osim.Inertia(0, 0, 0))
    links.append(link)
    model.addBody(link)

In [4]:
joints = []
prev_link = ground
for i in range(len(links)):
    next_link = links[i]
    joint = osim.PinJoint(f"joint_{i}",
                         prev_link, # PhysicalFrame
                         osim.Vec3(0),
                         osim.Vec3(0),
                         next_link, # PhysicalFrame
                         osim.Vec3(0, 1, 0),
                         osim.Vec3(0))
    prev_link = links[i]
    joints.append(joint)
    model.addJoint(joint)
    

In [5]:
actuators = []
prev_link = ground
for i in range(len(links)):
    next_link = links[i]
    act = osim.TorqueActuator(
        prev_link,
        next_link,
        osim.Vec3(0, 0, 1),
    )
    act.setOptimalForce(0.0)
    actuators.append(act)
    model.addForce(act)
    

In [6]:
brain = osim.PrescribedController()
func = osim.Constant(1.0)
for i in range(len(links)):
    act = actuators[i]
    brain.addActuator(act)
    brain.prescribeControlForActuator(i, func)
model.addController(brain)

In [7]:
bodyGeometry = osim.Ellipsoid(0.1, 0.5, 0.1)
bodyGeometry.setColor(osim.Vec3(0.5)) # Gray

link_frames = []

for i in range(len(links)):
    link = links[i]
    
    link_frame = osim.PhysicalOffsetFrame()
    link_frame.setName(f"linkCenter_{i}")
    link_frame.setParentFrame(link)
    link_frame.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
    link.addComponent(link_frame)
    link_frame.attachGeometry(bodyGeometry.clone())
    link_frames.append(link_frame)
    

In [9]:
contact_geom = osim.ContactHalfSpace(
    osim.Vec3(0, 0, 0),
    osim.Vec3(0, 0, -1.5707963267949),
    ground,
    "platform"
)
model.addContactGeometry(contact_geom)

for i in range(len(links)):
    link = links[i]
    radius = 0.1
    name = f"contact_sphere_{i}"
    geometry = osim.ContactSphere(radius, osim.Vec3(0, 0, 0), link, name)
    model.addContactGeometry(geometry)
    
    force = osim.HuntCrossleyForce()
    force.setStiffness(1e10)
    force.setDissipation(1)
    force.setStaticFriction(0)
    force.setDynamicFriction(0)
    force.setViscousFriction(0)
    force.addGeometry(contact_geom.getName())
    force.addGeometry(geometry.getName())
    model.addForce(force)
    

In [31]:
state = model.initSystem()

In [32]:
jointSet = model.getJointSet()
for i in range(jointSet.getSize()):
    joint = jointSet.get(i)
    print(joint.getCoordinate().getValue(state))

0.0
0.0
0.0
0.0
0.0


In [33]:
jointSet.get(0).getCoordinate().setValue(state, 160)

In [34]:
# for i in range(jointSet.getSize()):
#     jointSet.get(i).getCoordinate().setValue(state, 60)

In [35]:
#state = model.initSystem()
viz = model.updVisualizer().updSimbodyVisualizer()
viz.setBackgroundColor(osim.Vec3(0)) # white
viz.setGroundHeight(0)

# ---------------------------------------------------------------------------
# Simulate.
# ---------------------------------------------------------------------------

manager = osim.Manager(model)
state.setTime(0)
manager.initialize(state)
state = manager.integrate(0.001)

In [37]:

res = {}

## Joints
res["joint_pos"] = {}
res["joint_vel"] = {}
res["joint_acc"] = {}
jointSet = model.getJointSet()
for i in range(jointSet.getSize()):
    joint = jointSet.get(i)
    name = joint.getName()
    res["joint_pos"][name] = [joint.get_coordinates(i).getValue(state) for i in range(joint.numCoordinates())]
    res["joint_vel"][name] = [joint.get_coordinates(i).getSpeedValue(state) for i in range(joint.numCoordinates())]
    res["joint_acc"][name] = [joint.get_coordinates(i).getAccelerationValue(state) for i in range(joint.numCoordinates())]
res

{'joint_pos': {'joint_0': [159.9999989240988],
  'joint_1': [1.0759054761896635e-06],
  'joint_2': [-4.2890213495609216e-12],
  'joint_3': [8.141635513917815e-22],
  'joint_4': [-7.401486830834377e-23]},
 'joint_vel': {'joint_0': [-0.002151810952437256],
  'joint_1': [0.002151828108606885],
  'joint_2': [-1.715619698690792e-08],
  'joint_3': [2.7357819713339872e-14],
  'joint_4': [-7.401486830834377e-20]},
 'joint_acc': {'joint_0': [-2.1518452648974726],
  'joint_1': [2.1518967336590644],
  'joint_2': [-5.146892572982509e-05],
  'joint_3': [1.641375924066324e-10],
  'joint_4': [8.881784197001252e-16]}}

In [36]:
model.printToXML('/mnt/data/Linux/VMs/VMs/Win10/shared/5-link.osim')

True

In [8]:
# arm = osim.Model()
# arm.setName("bicep_curl")
# arm.setUseVisualizer(True)

# # ---------------------------------------------------------------------------
# # Create two links, each with a mass of 1 kg, centre of mass at the body's
# # origin, and moments and products of inertia of zero.
# # ---------------------------------------------------------------------------

# humerus = osim.Body("humerus",
#                     1.0,
#                     osim.Vec3(0),
#                     osim.Inertia(0, 0, 0))
# radius = osim.Body("radius",
#                    1.0,
#                    osim.Vec3(0),
#                    osim.Inertia(0, 0, 0))

# # ---------------------------------------------------------------------------
# # Connect the bodies with pin joints. Assume each body is 1m long.
# # ---------------------------------------------------------------------------

# shoulder = osim.PinJoint("shoulder",
#                          arm.getGround(), # PhysicalFrame
#                          osim.Vec3(0),
#                          osim.Vec3(0),
#                          humerus, # PhysicalFrame
#                          osim.Vec3(0, 1, 0),
#                          osim.Vec3(0))

# elbow = osim.PinJoint("elbow",
#                       humerus, # PhysicalFrame
#                       osim.Vec3(0),
#                       osim.Vec3(0),
#                       radius, # PhysicalFrame
#                       osim.Vec3(0, 1, 0),
#                       osim.Vec3(0))

# # ---------------------------------------------------------------------------
# # Add a muscle that flexes the elbow (actuator for robotics people).
# # ---------------------------------------------------------------------------

# biceps = osim.Millard2012EquilibriumMuscle("biceps",  # Muscle name
#                                            200.0,  # Max isometric force
#                                            0.6,  # Optimal fibre length
#                                            0.55,  # Tendon slack length
#                                            0.0)  # Pennation angle
# biceps.addNewPathPoint("origin",
#                        humerus,
#                        osim.Vec3(0, 0.8, 0))

# biceps.addNewPathPoint("insertion",
#                        radius,
#                        osim.Vec3(0, 0.7, 0))

# # ---------------------------------------------------------------------------
# # Add a controller that specifies the excitation of the muscle.
# # ---------------------------------------------------------------------------

# brain = osim.PrescribedController()
# brain.addActuator(biceps)
# brain.prescribeControlForActuator("biceps",
#                                   osim.StepFunction(0.5, 3.0, 0.3, 1.0))

# # ---------------------------------------------------------------------------
# # Build model with components created above.
# # ---------------------------------------------------------------------------

# arm.addBody(humerus)
# arm.addBody(radius)
# arm.addJoint(shoulder) # Now required in OpenSim4.0
# arm.addJoint(elbow)
# arm.addForce(biceps)
# arm.addController(brain)

# # ---------------------------------------------------------------------------
# # Add a console reporter to print the muscle fibre force and elbow angle.
# # ---------------------------------------------------------------------------

# # We want to write our simulation results to the console.
# reporter = osim.ConsoleReporter()
# reporter.set_report_time_interval(1.0)
# reporter.addToReport(biceps.getOutput("fiber_force"))
# elbow_coord = elbow.getCoordinate().getOutput("value")
# reporter.addToReport(elbow_coord, "elbow_angle")
# arm.addComponent(reporter)

# # ---------------------------------------------------------------------------
# # Add display geometry. 
# # ---------------------------------------------------------------------------

# bodyGeometry = osim.Ellipsoid(0.1, 0.5, 0.1)
# bodyGeometry.setColor(osim.Vec3(0.5)) # Gray
# humerusCenter = osim.PhysicalOffsetFrame()
# humerusCenter.setName("humerusCenter")
# humerusCenter.setParentFrame(humerus)
# humerusCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
# humerus.addComponent(humerusCenter)
# humerusCenter.attachGeometry(bodyGeometry.clone())

# radiusCenter = osim.PhysicalOffsetFrame()
# radiusCenter.setName("radiusCenter")
# radiusCenter.setParentFrame(radius)
# radiusCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
# radius.addComponent(radiusCenter)
# radiusCenter.attachGeometry(bodyGeometry.clone())

# # ---------------------------------------------------------------------------
# # Configure the model.
# # ---------------------------------------------------------------------------

# state = arm.initSystem()
# # Fix the shoulder at its default angle and begin with the elbow flexed.
# shoulder.getCoordinate().setLocked(state, True)
# elbow.getCoordinate().setValue(state, 0.5 * osim.SimTK_PI)
# arm.equilibrateMuscles(state)


In [5]:

# # ---------------------------------------------------------------------------
# # Configure the visualizer.
# # ---------------------------------------------------------------------------

# viz = arm.updVisualizer().updSimbodyVisualizer()
# viz.setBackgroundColor(osim.Vec3(0)) # white
# viz.setGroundHeight(-2)

# # ---------------------------------------------------------------------------
# # Simulate.
# # ---------------------------------------------------------------------------

# manager = osim.Manager(arm)
# state.setTime(0)
# manager.initialize(state)
# state = manager.integrate(10.0)